In [2]:
from IPython.core.debugger import prompt
from langchain.chains.qa_with_sources.stuff_prompt import template

from config import *
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek

llm = ChatOpenAI(base_url="https://api.deepseek.com/v1", model="deepseek-chat")

In [3]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator

In [4]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, model_validator

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [25]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables="query",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [37]:
# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | llm
output = prompt_and_model.invoke({"query": "Tell me a joke."})
output.pretty_print()

================================== Ai Message ==================================

```json
{
  "setup": "Why don't skeletons fight each other?",
  "punchline": "They don't have the guts."
}
```


In [36]:
# And a query intended to prompt a language model to populate the data structure.
chain = prompt | llm | parser
output = chain.invoke({"query": "Tell me a joke."})
output

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

In [41]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | llm | json_parser

In [42]:
list(json_chain.stream({"question": "Who invented the microscope?"}))

[{},
 {'answer': ''},
 {'answer': 'The'},
 {'answer': 'The invention'},
 {'answer': 'The invention of'},
 {'answer': 'The invention of the'},
 {'answer': 'The invention of the microscope'},
 {'answer': 'The invention of the microscope is'},
 {'answer': 'The invention of the microscope is often'},
 {'answer': 'The invention of the microscope is often attributed'},
 {'answer': 'The invention of the microscope is often attributed to'},
 {'answer': 'The invention of the microscope is often attributed to several'},
 {'answer': 'The invention of the microscope is often attributed to several individuals'},
 {'answer': 'The invention of the microscope is often attributed to several individuals,'},
 {'answer': 'The invention of the microscope is often attributed to several individuals, but'},
 {'answer': 'The invention of the microscope is often attributed to several individuals, but the'},
 {'answer': 'The invention of the microscope is often attributed to several individuals, but the first'},

In [43]:
list(chain.stream({"query": "Tell me a joke."}))

[Joke(setup="Why don't scientists trust atoms?", punchline=''),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')]